<a href="https://colab.research.google.com/github/ujs/MachineLearningProjects/blob/master/knn_iris_pd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Goal- to classify flowers based on sepal and petal attributes using k- nearest neighbours algorithm
#import all main libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import io

In [2]:
from google.colab import files
uploaded = files.upload()

Saving iris.csv to iris (1).csv


In [0]:
# Step1- Calculate Euclidean distance between 2 points
def euclid_dist(vector1,vector2):
  d = 0
  for a,b in zip(vector1, vector2):
    d += (a-b)**2
  dist = np.sqrt(d)
  return dist

In [9]:
# Test euclidean distance function
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
row0 = dataset[0]
for row in dataset:
	distance = euclid_dist(row0, row)
	print(distance)


0.0
1.3290173915275787
1.9494646655653247
1.5591439385540549
0.5356280721938492
4.952940611164215
2.7789902674782985
4.3312480380207
6.59862349695304
5.084885603993178


In [10]:
# Step 2-Get k nearest neighbours
def knn (vector, data, k):
  dist = []
  for i in data:
    dist.append(euclid_dist(vector,i))
  #print(dist)
  return sorted(dist)[1:k+1]

#test

knn ([7.627531214,2.759262235,1], dataset, 4)



[0.7507252672160873,
 1.2138653345155597,
 2.3910645897263616,
 3.1790022321304874]

In [0]:
# Step3- Predict the class

def predict_knn(vector, data, k):
  neighbours = knn(vector, data, k)
  classes = [neighbour[-1] for neighbour in neighbours]
  prediction = max(set(classes), key=classes.count)
  return prediction


In [0]:
#Implementation on the Iris dataset
main_df = pd.read_csv(io.BytesIO(uploaded['iris.csv']))
main_df = main_df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']]
df = main_df.copy()
df = df.drop(149, axis = 0)

#shuffling all rows in place
df = df.sample(frac=1)

df.sepal_length = df.sepal_length.astype(float)


In [75]:
#Data Preprocessing
from random import seed
from random import randrange
from csv import reader
  

#Mapping class labels to integer
def string_to_int(df,column):
  mapping = dict()
  column_values = [df.iloc[index][column] for index in range(len(df))]
  for value, key in enumerate(list(df[column].unique())):
    mapping[key] = value
  for index in range(len(df)):
    class_name = df.iloc[index][column]
    df.at[index, column] = mapping[class_name]
  return mapping


#test mapping
string_to_int(df,'class')

df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,4.9,3.0,1.4,0.2,0
1,4.7,3.2,1.3,0.2,0
2,4.6,3.1,1.5,0.2,0
3,5.0,3.6,1.4,0.2,0
4,5.4,3.9,1.7,0.4,0


In [0]:
#Normalize data from values 0 to 1
def range_min(column):
  max_val = df[column].max()
  min_val = df[column].min()
  range_val = max_val - min_val
  return min_val, range_val

#testing
range_min('sepal_length')

#Calculate range and min for a column
def range_min(column):
  max_val = df[column].max()
  min_val = df[column].min()
  range_val = max_val - min_val
  return min_val, range_val

def normalize(df):
  '''input is data frame'''
  for index in range(len(df)):
    for column in df.columns:
      df.at[index,column] = (df.iloc[index][column] - range_min(column)[0])/range_min(column)[1]


normalize(df)

In [0]:
from sklearn import model_selection
from sklearn.model_selection import KFold
# Split a dataset into k folds
def cross_validation_split(df, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(df) / n_folds)
	for _ in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [86]:
kfold = model_selection.KFold(n_splits=5, random_state=100)
model_kfold = LogisticRegression()
results_kfold = model_selection.cross_val_score(model_kfold, x1, y1, cv=kfold)
print("Accuracy: %.2f%%" % (results_kfold.mean()*100.0)) 

39        Iris-setosa
4         Iris-setosa
48        Iris-setosa
108    Iris-virginica
121    Iris-virginica
Name: class, dtype: object